## DOT_Traffic_Speeds_NBE-3.csv
## DOT_Traffic_Speeds_NBE-4.csv

##### I cannot upload the dataset to Jupyter Hub because the size of the csv is too large. Instead, I first clean the data using Jupyter notebook and then upload the cleaned dataset.

In [1]:
import pandas as pd
import datetime
import csv

##### read in the dataset

In [2]:
traffic_speed_18 = pd.read_csv("DOT_Traffic_Speeds_NBE-3.csv")
traffic_speed_18.to_pickle("traffic_speed_18.pkl")
df_pickle_traffic_speed_18 = pd.read_pickle("traffic_speed_18.pkl")

traffic_speed_19 = pd.read_csv("DOT_Traffic_Speeds_NBE-4.csv")
traffic_speed_19.to_pickle("traffic_speed_19.pkl")
df_pickle_traffic_speed_19 = pd.read_pickle("traffic_speed_19.pkl")

In [3]:
df_traffic_speed = df_pickle_traffic_speed_18.append([df_pickle_traffic_speed_19])

In [4]:
df_traffic_speed.tail(5)

,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME
218125,217,37.28,274,0,08/30/2019 11:58:11 PM,4616328,"40.71141,-73.97866 40.712,-73.97793 40.712661,...",w~mwFjisbMc@}HkCe^gBoZe@iQe@mJSaBYmA_A{BuBqCcC...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616328,Manhattan,FDR N Catherine Slip - 25th St
218126,221,30.44,332,0,08/30/2019 11:58:11 PM,4616333,"40.736971,-73.97477 40.73651,-73.975011 40.735...",amswFheobMzAn@nBHdD]rE{@hGaAjEq@bAc@pAs@x@}@~A...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616333,Manhattan,FDR S 25th St - Catherine Slip
218127,1,9.94,541,0,08/30/2019 11:58:11 PM,4616337,"40.74047,-74.009251 40.74137,-74.00893 40.7431...",}btwFx\\\\\\\\\\\\\\|ubMsD_AgJcAcR{ByJ_AsBFiEb...,BBBBBBBBBBBBB,NYC_DOT_LIC,4616337,Manhattan,11th ave n ganservoort - 12th ave @ 40th st
218128,223,30.44,86,0,08/30/2019 11:58:11 PM,4616341,"40.70908,-73.9959 40.70895,-73.996941 40.70882...",w~mwFjisbMXnEXdCh@bCf@zAd@jA^r@,BBBBBBB,NYC_DOT_LIC,4616341,Manhattan,FDR S Catherine Slip - BKN Bridge Manhattan Side
218129,145,37.28,70,0,08/30/2019 11:58:11 PM,4616342,"40.7081105,-73.99944 40.7084705,-73.99884 40.7...",uxmwFn_tbMgAwBi@eBc@sB[qC[wE,BBBBBB,NYC_DOT_LIC,4616342,Manhattan,BKN Bridge Manhattan Side - FDR N Catherine Slip


### Preprocessing
###### 1. Data Cleaning has done using the filter provided on the web source. 
###### 2. The date attribute need to be separate into dates and time. The dates attribute should be in the same format with TLC dataset. 
###### 3. Attributes like ENCODED-POLY-LINE-LVLS, ID, LINK-ID, LINK-POINTS, ENCODED-POLY-LINE, OWNER, TRANSCOM-ID, BOROUGH, LINK-NAME, STATUS, ID are all irrelevant to this study. They can all be deleted.
###### 4. Removed outliers of unusal low and high speed by calculating a IQR. Filtered instances higher or lower than 1.5IQR.
###### 5. Create a new attribute which compute the average daily traffic speed of Manhattan.

In [5]:
#Easy and straightforward processing steps

#Choosing instances with status equals to 0
df_traffic_speed = df_traffic_speed[df_traffic_speed["STATUS"]==0]

#Choosing instances with nonzero travel time
df_traffic_speed = df_traffic_speed[df_traffic_speed["TRAVEL_TIME"]!=0]

#Choosing instances with nonzero average speed
df_traffic_speed = df_traffic_speed[df_traffic_speed["SPEED"]!=0]

#Choosing instances owned by NYC_DOT_LIC to control
df_traffic_speed = df_traffic_speed[df_traffic_speed["OWNER"]=="NYC_DOT_LIC"]

#Choosing instances only within Manhattan, the studied area
df_traffic_speed = df_traffic_speed[df_traffic_speed["BOROUGH"]=="Manhattan"]

In [6]:
#format datetime
df_traffic_speed['DATA_AS_OF']=pd.to_datetime(df_traffic_speed['DATA_AS_OF'], format='%m/%d/%Y %H:%M:%S %p')


#Splitting timestamp column into separate date and time columns
df_traffic_speed['Dates'] = pd.to_datetime(df_traffic_speed['DATA_AS_OF']).dt.date
df_traffic_speed['Time'] = pd.to_datetime(df_traffic_speed['DATA_AS_OF']).dt.time

In [7]:
#have a better understanding of the data
df_traffic_speed.dtypes

ID                                 int64
SPEED                            float64
TRAVEL_TIME                        int64
STATUS                             int64
DATA_AS_OF                datetime64[ns]
LINK_ID                            int64
LINK_POINTS                       object
ENCODED_POLY_LINE                 object
ENCODED_POLY_LINE_LVLS            object
OWNER                             object
TRANSCOM_ID                        int64
BOROUGH                           object
LINK_NAME                         object
Dates                             object
Time                              object
dtype: object

In [8]:
df_traffic_speed.tail(5)

,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME,Dates,Time
218125,217,37.28,274,0,2019-08-30 11:58:11,4616328,"40.71141,-73.97866 40.712,-73.97793 40.712661,...",w~mwFjisbMc@}HkCe^gBoZe@iQe@mJSaBYmA_A{BuBqCcC...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616328,Manhattan,FDR N Catherine Slip - 25th St,2019-08-30,11:58:11
218126,221,30.44,332,0,2019-08-30 11:58:11,4616333,"40.736971,-73.97477 40.73651,-73.975011 40.735...",amswFheobMzAn@nBHdD]rE{@hGaAjEq@bAc@pAs@x@}@~A...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616333,Manhattan,FDR S 25th St - Catherine Slip,2019-08-30,11:58:11
218127,1,9.94,541,0,2019-08-30 11:58:11,4616337,"40.74047,-74.009251 40.74137,-74.00893 40.7431...",}btwFx\\\\\\\\\\\\\\|ubMsD_AgJcAcR{ByJ_AsBFiEb...,BBBBBBBBBBBBB,NYC_DOT_LIC,4616337,Manhattan,11th ave n ganservoort - 12th ave @ 40th st,2019-08-30,11:58:11
218128,223,30.44,86,0,2019-08-30 11:58:11,4616341,"40.70908,-73.9959 40.70895,-73.996941 40.70882...",w~mwFjisbMXnEXdCh@bCf@zAd@jA^r@,BBBBBBB,NYC_DOT_LIC,4616341,Manhattan,FDR S Catherine Slip - BKN Bridge Manhattan Side,2019-08-30,11:58:11
218129,145,37.28,70,0,2019-08-30 11:58:11,4616342,"40.7081105,-73.99944 40.7084705,-73.99884 40.7...",uxmwFn_tbMgAwBi@eBc@sB[qC[wE,BBBBBB,NYC_DOT_LIC,4616342,Manhattan,BKN Bridge Manhattan Side - FDR N Catherine Slip,2019-08-30,11:58:11


In [9]:
#Delete the useless attributes
df_traffic_speed = df_traffic_speed[["SPEED", "TRAVEL_TIME", "Dates", "Time"]]

In [10]:
df_traffic_speed.tail(5)

,SPEED,TRAVEL_TIME,Dates,Time
218125,37.28,274,2019-08-30,11:58:11
218126,30.44,332,2019-08-30,11:58:11
218127,9.94,541,2019-08-30,11:58:11
218128,30.44,86,2019-08-30,11:58:11
218129,37.28,70,2019-08-30,11:58:11


In [11]:
df_traffic_speed.to_csv("Traffic_speed_cleaned_all")

###### Have a overall view of the data. The data has fairly good

In [12]:
subset_data_1 = ['SPEED', 'TRAVEL_TIME']

df_traffic_speed[subset_data_1].describe()

,SPEED,TRAVEL_TIME
count,438718.000000,438718.000000
mean,25.414027,351.761232
std,13.052201,302.190623
min,0.620000,42.000000
25%,14.910000,165.000000
50%,22.990000,292.000000
75%,36.030000,447.000000
max,61.510000,8903.000000


##### Filter all the outliers, using IQR
##### Perform fltering on SPEED and TRAVEL_TIME

In [15]:
# Select the first quantile
q1_speed = df_traffic_speed['SPEED'].quantile(.25)

# Select the third quantile
q3_speed = df_traffic_speed['SPEED'].quantile(.75)

IQR_speed = q3_speed - q1_speed

# Create a mask inbeetween q1 & q3
mask_speed = df_traffic_speed['SPEED'].between(q1_speed - 1.5*IQR_speed, q3_speed + 1.5*IQR_speed, inclusive=True)

# Filtering the initial dataframe with a mask
df_traffic_speed = df_traffic_speed.loc[mask_speed==True]


In [19]:
# Select the first quantile
q1_travel_time = df_traffic_speed['TRAVEL_TIME'].quantile(.25)

# Select the third quantile
q3_travel_time = df_traffic_speed['TRAVEL_TIME'].quantile(.75)

IQR_travel_time = q3_travel_time - q1_travel_time

# Create a mask inbeetween q1 & q3
mask_travel_time = df_traffic_speed['TRAVEL_TIME'].between(q1_travel_time - 1.5*IQR_travel_time, 
                                                           q3_travel_time + 1.5*IQR_travel_time, inclusive=True)

# Filtering the initial dataframe with a mask
df_traffic_speed = df_traffic_speed.loc[mask_travel_time==True]

##### From here, groupby the data by dates

In [27]:
new_df = df_traffic_speed.groupby([df_traffic_speed['Dates']])['SPEED', "TRAVEL_TIME"].mean()

##### At last generate a new csv for the data.

In [29]:
new_df.to_csv("new_df_traffic")